In [2]:
import os
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed
from isimip_client.client import ISIMIPClient

# Download isimip data

In [4]:
# ------------------ تنظیمات ------------------

variables = ["pr", "tas"]

scenarios = ["ssp126", "ssp370", "ssp585"]

models = [
    "gfdl-esm4",
    "ipsl-cm6a-lr",
    "mpi-esm1-2-hr",
    "mri-esm2-0",
    "ukesm1-0-ll"
]

WATERSHED_SHP_PATH = r"E:\Term2\WERI\thesis_AB\Research\code\shp\Mashhad.shp"
base_download_dir = "isimip"

os.makedirs(base_download_dir, exist_ok=True)

client = ISIMIPClient()

# ------------------ خواندن shapefile ------------------

gdf = gpd.read_file(WATERSHED_SHP_PATH)
lonmin, latmin, lonmax, latmax = gdf.total_bounds

# ------------------ توابع ------------------

def get_filtered_files(var, model, sce):
    response = client.datasets(
        simulation_round="ISIMIP3b",
        product="InputData",
        climate_forcing=model,
        climate_scenario=sce,
        climate_variable=var
    )

    file_paths = []
    for dataset in response["results"]:
        for f in dataset["files"]:
            file_paths.append(f["path"])

    return file_paths


def download_for_site(var, model, sce):
    var_dir = os.path.join(base_download_dir, var, model, sce)
    os.makedirs(var_dir, exist_ok=True)

    if os.listdir(var_dir):
        print(f"⏭️ Skipping {var} | {model} | {sce}")
        return

    try:
        files = get_filtered_files(var, model, sce)

        cutout = client.cutout(
            files,
            [latmin, latmax, lonmin, lonmax],
            poll=10
        )

        client.download(
            cutout["file_url"],
            path=var_dir,
            validate=False,
            extract=False
        )

        print(f"✅ Downloaded {var} | {model} | {sce}")

    except Exception as e:
        print(f"⚠️ Error {var} | {model} | {sce}: {e}")

# ------------------ اجرای موازی ------------------

max_workers = 8
futures = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    for var in variables:
        for model in models:
            for sce in scenarios:
                futures.append(
                    executor.submit(download_for_site, var, model, sce)
                )

    for future in as_completed(futures):
        future.result()

print("\n🎯 All downloads complete.")


⏭️ Skipping pr | gfdl-esm4 | ssp126
⏭️ Skipping pr | gfdl-esm4 | ssp370
⏭️ Skipping pr | gfdl-esm4 | ssp585
⏭️ Skipping pr | ipsl-cm6a-lr | ssp126
⏭️ Skipping pr | ipsl-cm6a-lr | ssp585
⏭️ Skipping pr | mpi-esm1-2-hr | ssp126
⏭️ Skipping pr | mpi-esm1-2-hr | ssp370
⏭️ Skipping pr | ipsl-cm6a-lr | ssp370
⏭️ Skipping pr | mri-esm2-0 | ssp126
⏭️ Skipping pr | mri-esm2-0 | ssp370
⏭️ Skipping pr | ukesm1-0-ll | ssp126
⏭️ Skipping pr | ukesm1-0-ll | ssp585
⏭️ Skipping tas | gfdl-esm4 | ssp370
⏭️ Skipping tas | gfdl-esm4 | ssp585
⏭️ Skipping tas | ipsl-cm6a-lr | ssp126
⏭️ Skipping tas | mpi-esm1-2-hr | ssp370
⏭️ Skipping tas | ipsl-cm6a-lr | ssp585
⏭️ Skipping tas | ipsl-cm6a-lr | ssp370
⏭️ Skipping tas | mpi-esm1-2-hr | ssp126
⏭️ Skipping tas | mri-esm2-0 | ssp126
⏭️ Skipping tas | mpi-esm1-2-hr | ssp585
⏭️ Skipping tas | mri-esm2-0 | ssp370
⏭️ Skipping tas | mri-esm2-0 | ssp585
⏭️ Skipping tas | ukesm1-0-ll | ssp126
⏭️ Skipping tas | ukesm1-0-ll | ssp370
⏭️ Skipping tas | ukesm1-0-ll | ssp5

In [ ]:
# plot the time series
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

fig, axes = plt.subplots(nrows=len(points))
for index, point in enumerate(points):
    lat, lon, download_path = point
    frames = []
    for file in sorted(Path(download_path).iterdir()):
        if file.suffix == '.csv':
            frames.append(pd.read_csv(file, names=['date', 'pr'], index_col='date', parse_dates=True))
    
    df = pd.concat(frames)
    df.plot(ax=axes[index], title=download_path, xlim=('2020-01-01', '2030-01-01'))

plt.gcf().set_size_inches(20, 40)